<a href="https://colab.research.google.com/github/MdAfzalkhan20112000/EHRS-USING-LLM/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ===============================
# EHR + Mini LLM Demo (20 patients)
# ===============================

!pip install gradio scikit-learn pandas

import random
import pandas as pd
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -------------------------------
# Step 1: Create synthetic dataset (20 patients)
# -------------------------------

patients = [
    {"id": "P001", "name": "John Doe", "age": 45,
     "diagnoses": ["Hypertension", "Type 2 Diabetes"],
     "medications": ["Metformin", "Lisinopril"],
     "labs": {"HbA1c": "7.5%", "Cholesterol": "220 mg/dL"},
     "notes": "Moderate diabetes control. Needs better diet and exercise."},

    {"id": "P002", "name": "Jane Smith", "age": 60,
     "diagnoses": ["Asthma"],
     "medications": ["Albuterol inhaler"],
     "labs": {"FEV1": "65% predicted"},
     "notes": "Reports wheezing at night. Uses inhaler daily."},

    {"id": "P003", "name": "Ali Khan", "age": 35,
     "diagnoses": ["Anemia"],
     "medications": ["Iron supplements"],
     "labs": {"Hemoglobin": "10.2 g/dL"},
     "notes": "Fatigue and weakness. Responding to iron therapy."},

    {"id": "P004", "name": "Michael Johnson", "age": 52,
     "diagnoses": ["Coronary Artery Disease"],
     "medications": ["Aspirin", "Atorvastatin"],
     "labs": {"Cholesterol": "250 mg/dL"},
     "notes": "History of MI. Ex-smoker."},

    {"id": "P005", "name": "Sophia Lee", "age": 29,
     "diagnoses": ["Migraine"],
     "medications": ["Sumatriptan"],
     "labs": {"MRI": "Normal"},
     "notes": "Migraines 1-2 per month, responds to triptan."},

    {"id": "P006", "name": "David Brown", "age": 61,
     "diagnoses": ["Arthritis"],
     "medications": ["Ibuprofen"],
     "labs": {"CRP": "10 mg/L"},
     "notes": "Osteoarthritis of knees with chronic pain."},

    {"id": "P007", "name": "Emma Wilson", "age": 50,
     "diagnoses": ["Breast Cancer (Remission)"],
     "medications": ["Tamoxifen"],
     "labs": {"Mammogram": "Clear"},
     "notes": "In remission, annual follow-up imaging stable."},

    {"id": "P008", "name": "James Miller", "age": 40,
     "diagnoses": ["Obesity", "Sleep Apnea"],
     "medications": ["CPAP Therapy"],
     "labs": {"BMI": "32"},
     "notes": "Uses CPAP nightly; weight loss advised."},

    {"id": "P009", "name": "Olivia Davis", "age": 33,
     "diagnoses": ["Thyroid Disorder"],
     "medications": ["Levothyroxine"],
     "labs": {"TSH": "4.2 mIU/L"},
     "notes": "On stable dose of levothyroxine; energy improving."},

    {"id": "P010", "name": "William Garcia", "age": 47,
     "diagnoses": ["Chronic Kidney Disease"],
     "medications": ["Erythropoietin"],
     "labs": {"Creatinine": "2.0 mg/dL"},
     "notes": "Stage 3 CKD; follow renal labs regularly."},

    {"id": "P011", "name": "Isabella Martinez", "age": 55,
     "diagnoses": ["Type 2 Diabetes"],
     "medications": ["Insulin"],
     "labs": {"HbA1c": "8.1%"},
     "notes": "Requires insulin titration; monitors blood sugars at home."},

    {"id": "P012", "name": "Ethan Hernandez", "age": 30,
     "diagnoses": ["Anxiety Disorder"],
     "medications": ["Sertraline"],
     "labs": {"GAD-7": "10"},
     "notes": "On SSRI with good response; counselling ongoing."},

    {"id": "P013", "name": "Mia Thompson", "age": 43,
     "diagnoses": ["Rheumatoid Arthritis"],
     "medications": ["Methotrexate"],
     "labs": {"CRP": "15 mg/L"},
     "notes": "DMARD therapy in progress; joint pain improving."},

    {"id": "P014", "name": "Alexander White", "age": 60,
     "diagnoses": ["Parkinson's Disease"],
     "medications": ["Levodopa"],
     "labs": {"UPDRS": "25"},
     "notes": "Tremor and bradykinesia, on levodopa with benefit."},

    {"id": "P015", "name": "Charlotte Hall", "age": 25,
     "diagnoses": ["Anemia"],
     "medications": ["Iron Supplements"],
     "labs": {"Hemoglobin": "9.8 g/dL"},
     "notes": "Iron deficiency anemia, improving on therapy."},

    {"id": "P016", "name": "Benjamin Allen", "age": 49,
     "diagnoses": ["Gout"],
     "medications": ["Allopurinol"],
     "labs": {"Uric Acid": "8.2 mg/dL"},
     "notes": "Recurrent gout flares; advised urate lowering therapy."},

    {"id": "P017", "name": "Amelia Young", "age": 36,
     "diagnoses": ["PCOS"],
     "medications": ["Metformin"],
     "labs": {"Testosterone": "60 ng/dL"},
     "notes": "Irregular cycles; on metformin and lifestyle advice."},

    {"id": "P018", "name": "Henry King", "age": 70,
     "diagnoses": ["Alzheimer's Disease"],
     "medications": ["Donepezil"],
     "labs": {"MMSE": "22"},
     "notes": "Cognitive decline progressive; family support arranged."},

    {"id": "P019", "name": "Evelyn Scott", "age": 42,
     "diagnoses": ["Chronic Back Pain"],
     "medications": ["Gabapentin"],
     "labs": {"MRI": "Mild disc bulge"},
     "notes": "Chronic lumbar pain; physiotherapy advised."},

    {"id": "P020", "name": "Daniel Green", "age": 54,
     "diagnoses": ["COPD"],
     "medications": ["Tiotropium Inhaler"],
     "labs": {"FEV1": "60% predicted"},
     "notes": "Ex-smoker; breathlessness on exertion."},
]

ehr_df = pd.DataFrame(patients)

# -------------------------------
# Step 2: Build mini LLM system
# -------------------------------

# Build TF-IDF vectors for each patient record
docs = []
ids = []
for p in patients:
    context = f"""
    Name: {p['name']}, Age: {p['age']}
    Diagnoses: {', '.join(p['diagnoses'])}
    Medications: {', '.join(p['medications'])}
    Labs: {p['labs']}
    Notes: {p['notes']}
    """
    docs.append(context)
    ids.append(p["id"])

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

def query_ehr(patient_id, question):
    if patient_id not in ids:
        return "Invalid patient ID."

    idx = ids.index(patient_id)
    record = docs[idx]
    p = patients[idx]

    q = question.lower()

    # Keyword-based answers (like a mini-LLM)
    if "medication" in q or "drug" in q:
        return f"Medications: {', '.join(p['medications'])}"
    elif "diagnose" in q or "disease" in q or "condition" in q:
        return f"Diagnoses: {', '.join(p['diagnoses'])}"
    elif "lab" in q or "result" in q:
        return f"Labs: {p['labs']}"
    elif "note" in q or "summary" in q or "history" in q:
        return f"Notes: {p['notes']}"
    elif "who has" in q:
        # Global search across patients
        found = []
        cond = q.replace("who has", "").strip().lower()
        for pat in patients:
            if any(cond in d.lower() for d in pat["diagnoses"]):
                found.append(f"{pat['id']} ({pat['name']})")
        return f"Patients with {cond}: {', '.join(found)}" if found else f"No patients found with {cond}"
    else:
        # Fallback: cosine similarity retrieval
        q_vec = vectorizer.transform([question])
        sim = cosine_similarity(q_vec, X[idx])
        return f"Based on record: {record.strip()}"

# -------------------------------
# Step 3: Gradio GUI
# -------------------------------

def ehr_chat(patient_id, question):
    return query_ehr(patient_id, question)

dropdown = gr.Dropdown(choices=ids, label="Select Patient ID", value="P001")
textbox = gr.Textbox(label="Ask a Question")
output = gr.Textbox(label="Answer")

demo = gr.Interface(
    fn=ehr_chat,
    inputs=[dropdown, textbox],
    outputs=output,
    title="Electronic Health Record (EHR) + Mini LLM Demo",
    description="Ask questions about synthetic patient records (medications, diagnoses, labs, notes, history)."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c2b74d113b3116d83b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
